# Prerequisites

Run ElasticSearch in Docker
```bash
docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3
```

Q1. Running Elastic 

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

Check if ElasticSearch is running

In [2]:
!curl localhost:9200

{
  "name" : "d9dd33b76d2f",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "Pg-hhDDyQG6d1RjQarpknA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


# Import

In [1]:
import json
from elasticsearch import Elasticsearch
from openai import OpenAI
from tqdm.auto import tqdm # to show progress bar

/Users/aln/Library/CloudStorage/OneDrive-Personal/Personal/Dev/llmzoomcamp01/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Implement

In [5]:
es_client = Elasticsearch('http://localhost:9200')
client = OpenAI()

# Getting the data

Now let's get the FAQ data. You can run this snippet:

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

# Create index in ElasticSearch

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "llmzoomcamp01-homework"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'llmzoomcamp01-homework'})

In [7]:
for doc in tqdm(documents):
    es_client.index(index=index_name, body=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

100%|██████████| 948/948 [00:03<00:00, 297.76it/s]


## Q3. Searching

Now let's search in our index. 

We will execute a query "How do I execute a command in a running docker container?". 

Use only `question` and `text` fields and give `question` a boost of 4, and use `"type": "best_fields"`.

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [8]:
query = 'How do I execute a command in a running docker container?'

In [13]:
def elastic_search(query, search_query):
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
        result_docs.append(hit['_score'])
    
    return result_docs

In [14]:
search_query = {
    "size": 5, # return top 5 results
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # question field is 3 times important
                    "type": "best_fields"
                }
            }
        }
    }
}
result = elastic_search(query, search_query)
for r in result:
    print(json.dumps(r, indent=2))

{
  "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
  "section": "5. Deploying Machine Learning Models",
  "question": "How do I debug a docker container?",
  "course": "machine-learning-zoomcamp"
}
84.050095
{
  "text": "In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\nnetwork name (docker network)\npostgres related variables for pgcli\nHostname\nUsername\nPort\nDatabase name\n$ docker run -it --rm --network pg-network ai2ys/dockerized-pgcli:4.0.1\n175dd47cda07:/# pgcli -h pg-database -U root -p 5432 -d ny_taxi\nPassword for root:\nServer: PostgreS

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [15]:
search_query = {
    "size": 3, # return top 5 results
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # question field is 3 times important
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
result = elastic_search(query, search_query)
for r in result:
    print(json.dumps(r, indent=2))

{
  "text": "Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)",
  "section": "5. Deploying Machine Learning Models",
  "question": "How do I debug a docker container?",
  "course": "machine-learning-zoomcamp"
}
84.050095
{
  "text": "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  "section": "5. Deploying Machine Learning Models",
  "question": "How do I copy files from my local machine to docker 

## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. 

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (`\n\n`)
```python
context_template = """
Q: {question}
A: {text}
""".strip()
```

Now use the context you just created along with the "How do I execute a command in a running docker container?" question 
to construct a prompt using the template below:

```
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

What's the length of the resulting prompt? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [19]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def elastic_search(query, search_query):
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [20]:
query = "How do I execute a command in a running docker container?"
search_query = {
    "size": 3, # return top 5 results
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # question field is 3 times important
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
prompt = build_prompt(query=query, search_results=elastic_search(query, search_query))
print(prompt)
print(len(prompt))

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT: 
section: 5. Deploying Machine Learning Models
question: How do I debug a docker container?
answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

section: 5. Deploying Machine Learning Models
question: How do I copy files from my local machine to docker container?
answer: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can 

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```


In [21]:
!uv pip install tiktoken

Resolved 7 packages in 502ms                                                 m⠋ Resolving dependencies...                                                     
⠙ Downloading packages... (0/2)                                                 
⠙ Downloading packages... (0/2)--     0 B/278.33 kB                     
⠙ Downloading packages... (0/2)-- 2.76 kB/278.33 kB                     
⠙ Downloading packages... (0/2)-- 6.89 kB/278.33 kB                     
⠙ Downloading packages... (0/2)-- 11.02 kB/278.33 kB                    
⠙ Downloading packages... (0/2)-- 15.16 kB/278.33 kB                    
⠙ Downloading packages... (0/2)-- 19.14 kB/278.33 kB                    
⠙ Downloading packages... (0/2)-- 23.27 kB/278.33 kB                    
⠙ Downloading packages... (0/2)-- 24.65 kB/278.33 kB                    
⠙ Downloading packages... (0/2)-- 28.79 kB/278.33 kB                    
⠙ Downloading packages... (0/2)-- 31.54 kB/278.33 kB                    
⠙ Downloading packages... (0/2

In [23]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [26]:
query_tokens = encoding.encode(prompt)
num_tokens = len(query_tokens)
print(f"Number of tokens: {num_tokens}")

Number of tokens: 356


In [27]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [30]:
def rag(query, search_query):
    search_results = elastic_search(query, search_query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [32]:
search_query = {
    "size": 3, # return top 5 results
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"], # question field is 3 times important
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}
rag(query, search_query)

'To execute a command in a running Docker container, you need to use the `docker exec` command. Here are the steps to do this:\n\n1. First, find the container ID of the running container by using the `docker ps` command.\n   ```sh\n   docker ps\n   ```\n\n2. Once you have the container ID, use the `docker exec` command to execute the desired command. For example, to start a bash session in the container, run:\n   ```sh\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of the container you want to execute the command in.'